In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

In [2]:
def Histograms(image, title):
    
    hist = cv2.calcHist(image, [0], None, [255], [0, 255])
    plt.ylabel('Pixels')
    plt.xlabel('Darkness')
    plt.title(title)
    plt.plot(hist)
    plt.xlim([0,256])
    plt.savefig(title + 'histogram.png')
    plt.clf()

In [5]:
images = {}
for x in os.listdir("Images"):
    images[x] = cv2.imread("Images/" + x)

In [15]:
class Window:
    
    def __init__(self, image, n, p):
        
        self.xlimit = image.shape[1]+n
        self.ylimit = image.shape[0]+n
        self.topleft = (0,0)
        self.bottomright = (n,n)
        self.prev = n
        self.height = n
        self.pace = (p, p)
        
        try:
            self.channels = image.shape[2]
        except:
            self.channels = 1
            
    def Position(self):
        
        return self.topleft, self.bottomright
    
    def NextPosition(self):
        
        #New Line Movement
        if(self.bottomright + self.pace)[0] >= (self.xlimit - self.height):
            return (0, self.topleft[1] + self.pace[1]), (self.height, self.bottomright[1] + self.pace[1])
        
        #Moving one pixel to the Right
        else:
            return (self.topleft[0] + self.pace[0], self.topleft[1]), (self.bottomright[0] + self.pace[0], self.bottomright[1])
        
    def NextMove(self):
        
        #Move Position of Window
        self.topleft, self.bottomright = self.NextPosition()
        return self.topleft, self.bottomright
    
    def Bounds(self, templeft = None, tempright = None):
        
        if templeft is None:
            templeft = self.topleft
        if tempright is None:
            tempright = self.bottomright 
            
        return tempright[0] <= self.xlimit and tempright[1] <= self.ylimit and templeft[0] >= 0 and templeft[1] >= 0

In [19]:
#Sliding Window Operations
show = False

window = Window(images["dababy.jpg"], 100, 50)
topleft, bottomright = window.Position()

image = cv2.rectangle(images["dababy.jpg"].copy(), topleft, bottomright, (0, 0, 255))
cv2.imwrite("Output/dababy.png", image, [cv2.IMWRITE_PNG_COMPRESSION, 0])

tempright, templeft = window.NextPosition()

while window.Bounds(bottomright):
    image = cv2.rectangle(images["dababy.jpg"].copy(), topleft, bottomright, (0, 0, 255))
    if show:
        cv2.imshow("Window", image)
        cv2.waitKey(int(1/35*1000))
        
    topleft, bottomright = window.NextMove()
    tempright = window.NextPosition()
    
cv2.destroyAllWindows()

In [23]:
class Kernel:
    
    def __init__(self, kernel, weight):
        
        self.kernel = kernel
        self.weight = weight 
        
    def FilterImage(self, image, pace = 1, window = None, axis = 0):
        
        img = []
        line = []
        
        if window is None:
            
            move = Window(image, self.kernel.shape[0], pace)
            
        else:
            
            image = window.getBounds(image)
            move = Window(image, self.kernel.shape[0], pace)
            
        topleft, _ = move.NextPosition()
        while move.Bounds(topleft):
            
            roi = move.getBounds(image)
            
            if move.newY():
                img.append(line)
                line = []
                
            line.append(self.filter(roi, axis, move.channels))
            
            move.NextMove()
            topleft, _ = move.NextPosition()
            
        return np.array(img)
        
    def Filter(self, roi, axis = 0, channels = 1):
        
        output = []
        
        if axis == 2:
            
            for i in range(channels):
                
                if channels == 1:
                    fil = self.kernel * roi
                
                else:
                    fil = self.kernel * roi[:, :, i]
                    
                filsum = fil.sum()
                
                if channels == 1:
                    fil = self.kernel.X * roi
                
                else:
                    
                    fil = self.kernel.X * roi[:, :, i]
                    
                filsum2 = fil.sum()
                
                output.append((((filsum**2) + (filsum2 **2)) ** (1/2)) * self.weight)
                
            return np.array(output)
        
        else:
            
            if axis == 0:
                kernel = self.kernel 
            else:
                self.kernel.X
                
            for i in range(channels):
                
                if channels == 1:
                    fil = kernel * roi
                
                else:
                    fil = kernel * roi[:, :, i]
                    
                output.append(fil.sum() * self.weight)
                
            return np.array(output)
        
    def newY(self):
        
        if self.prev == self.bottomright[1]:
            return False
        
        else:
            
            self.prev = self.bottomright[1]
            return True
        
    def getBounds(self, image):
        
        img = []
        
        for i in range(self.topleft[1], self.bottomright[1]):
            
            if i >= image.shape[0]:
                continue
                
            img.append(image[i][self.topleft[0]: self.bottomright[0]])
            
        if self.channels == 1:
            
            return np.resize(np.array(img), (self.height, self.height)) #self.height twice since square
        
        else:
            
            return np.resize(np.array(img), (self.height, self.height, self.channels))

In [26]:
class Sobel:
    
    def __init__(self, weight):
        
        self.kernel = Kernel(np.array([[-1 , 0, 1],
                                       [-2, 0 , 2],
                                       [-1, 0 , 1]]), weight)
        
    def FilterImage(self, image, pace = 1, window = None, axis = 2):
        return self.kernel.FilterImage(image, pace, window, axis)
    
class Gaussian:
    
    def __init__(self, size, weight):
        
        gauss = size // 2
        
        x = np.arange(0, size, 1, float)
        
        y = x[:, np.newaxis]
        
        tempx = tempy = size // 2
        self.kernel = Kernel(np.exp(-4 * np.log(2) * ((x - tempx) ** 2 + (y - tempy) ** 2) / gauss ** 2), weight)
        
    def FilterImage(self, image, pace = 1, window = None, axis = 0):
        return self.kernel.FilterImage(image, pace, window, axis)
    
class Bilinear:
    
    def __init__(self, weight):
        
        self.kernel = Kernel(np.array([[1, 2, 1],
                                       [2, 4, 2],
                                       [1, 2, 1]]), weight)
        
    def FilterImage(self, image, pace = 1, window = None, axis = 0):
        return self.kernel.FilterImage(image, pace, window, axis)